In [ ]:
!pip install spotipy tqdm psutil --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.9/268.9 kB 5.8 MB/s eta 0:00:00


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from tqdm import tqdm
import psutil, os, threading, time, gc, sys, random

In [ ]:
# Spotify setup
client_id = '72474caf37cd4fd7b860f677916a9ca5'
client_secret = '0f6196902ccf47b5bc2a41c7fb56854e'

auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
# Optimization function
def optimize_process():
    try:
        # Lower the priority (increase responsiveness for Colab)
        p = psutil.Process(os.getpid())
        p.nice(psutil.IDLE_PRIORITY_CLASS if os.name == 'nt' else 19)
        print("Process priority set to low.")
    except Exception as e:
        print("Could not set process priority:", e)

In [ ]:
# Search songs from a keyword
def search_spotify_tracks(query: str, limit: int = 10):
    results = sp.search(q=query, type='track', limit=limit)
    tracks = []
    for item in tqdm(results['tracks']['items'], desc="Fetching Tracks"):
        track = {
            'name': item['name'],
            'artist': item['artists'][0]['name'],
            'url': item['external_urls']['spotify']
        }
        tracks.append(track)
    return tracks

In [ ]:
# Monitor system performance
def monitor_system():
    for _ in range(5):
        cpu = psutil.cpu_percent()
        mem = psutil.virtual_memory().percent
        print(f"CPU: {cpu}% | Memory: {mem}%")
        time.sleep(1)

In [ ]:
# Main function
def run_spotify_search():
    # Set process to low priority
    optimize_process()

    # Start system monitor
    threading.Thread(target=monitor_system, daemon=True).start()

    # Input from user
    query = input("What do you want to search for? (e.g. chill, genre:lofi, artist:Taylor Swift): ").strip()
    if not query:
        print("No input provided.")
        return

    # Start timer
    start = time.time()

    tracks = search_spotify_tracks(query)

    # Memory cleanup after the search
    gc.collect()

    # End timer
    end = time.time()

    # Result
    print(f"\nFound {len(tracks)} tracks:\n")
    for i, t in enumerate(tracks, 1):
        print(f"{i}. {t['name']} by {t['artist']}")
        print(f"   {t['url']}\n")

    # Show runtime
    print(f"\nRuntime: {end - start:.2f} seconds")

    gc.collect()
    print("Memory cleanup completed.")

run_spotify_search()

Process priority set to low.
CPU: 1.4% | Memory: 1.2%
CPU: 0.2% | Memory: 1.2%
CPU: 0.2% | Memory: 1.2%
CPU: 0.8% | Memory: 1.2%
CPU: 1.3% | Memory: 1.2%
What do you want to search for? (e.g. chill, genre:lofi, artist:Taylor Swift): chill


Fetching Tracks: 100%|██████████| 7/7 [00:00<00:00, 101592.14it/s]


Found 7 tracks:

1. Chill Bill (feat. J. Davi$ & Spooks) by Rob $tone
   https://open.spotify.com/track/5uDASfU19gDxSjW8cnCaBp

2. Chill Baby by SZA
   https://open.spotify.com/track/0qpdzfTxAkOREtvvGO5oew

3. Chill Guy (with Kyle Richh) by 41
   https://open.spotify.com/track/5koLyxl6fWprtuT1TKlZ35

4. Chill Bae by Lil Uzi Vert
   https://open.spotify.com/track/4ppKM7xnkSAwSyKqD4QTY4

5. Chillin' Like a Villain by Sofia Carson
   https://open.spotify.com/track/1rM0CnyUiiw6A9CHJRXjZA

6. Chillin' It by Cole Swindell
   https://open.spotify.com/track/0Utg7sirTPqrs6Cx0FbriQ

7. Chill by LISA
   https://open.spotify.com/track/1QIUF20HdqMA0CJvkBOHNb


Runtime: 0.23 seconds
Memory cleanup completed.
